In [1]:
# Abrindo os arquivos e formatando 
from sklearn.model_selection import train_test_split
from tensorflow import keras
import pandas as pd
import tensorflow as tf
import numpy as np
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

data = pd.read_csv('data.csv', sep = ',')

# Separando atributos e resultados
x = data.drop(columns = ['result'])
y = data['result']

In [2]:
# Normalizando
from sklearn.preprocessing import MinMaxScaler
from pickle import dump

normalizer = MinMaxScaler()
normalizerModel = normalizer.fit(x)
dump(normalizerModel, open('normalizerModel.pkl', 'wb')) 

x_normalized = normalizer.fit_transform(x)
y = pd.DataFrame(y)

In [3]:
# Balanceando
from imblearn.over_sampling import SMOTE
resampler = SMOTE(k_neighbors=4)
x_balanced, y_balanced = resampler.fit_resample(x_normalized, y)

In [4]:
# Separando dados para atributos e treinamento
x_train, x_test, y_train, y_test = train_test_split(x_balanced, y_balanced, test_size=0.1)

In [5]:
# Verificando os dados
print("Quantidade de Dados de Treino:" ,len(x_train))
print("Quantidade de Dados de Teste:" ,len(x_test))

print("Quantidade de Dados de Treino e dos atributos:" ,x_train.shape)
print("Quantidade de Dados de Teste e dos atributos" ,x_test.shape)

print("Quantidade de Saidas de Treino e coluna:" ,y_train.shape)
print("Quantidade de Saidas de Teste e coluna:" ,y_test.shape)

print("min: ", y_train.min())
print("max: ", y_test.max())

Quantidade de Dados de Treino: 2203
Quantidade de Dados de Teste: 245
Quantidade de Dados de Treino e dos atributos: (2203, 840)
Quantidade de Dados de Teste e dos atributos (245, 840)
Quantidade de Saidas de Treino e coluna: (2203, 1)
Quantidade de Saidas de Teste e coluna: (245, 1)
min:  result    0
dtype: int64
max:  result    2
dtype: int64


In [6]:
# Criando o modelo da rede neural
modelo = keras.Sequential([
  keras.layers.Dense(250, activation=tf.nn.relu, input_shape =(840, )),
  keras.layers.Dense(150, activation=tf.nn.relu),
  keras.layers.Dense(110, activation=tf.nn.relu),
  keras.layers.Dense(50, activation=tf.nn.relu),
  keras.layers.Dense(80, activation=tf.nn.relu),
  keras.layers.Dense(3, activation=tf.nn.softmax)
])
modelo.compile(loss ='sparse_categorical_crossentropy', optimizer ='adam', metrics =['accuracy'])

In [7]:
# Treinando o modelo
x = modelo.fit(x_train, y_train, epochs = 150, batch_size = 64)
test_loss, test_accuracy = modelo.evaluate(x_test, y_test)

Epoch 1/150
35/35 [==============================] - 0s 3ms/step - loss: 0.2291 - accuracy: 0.9378
Epoch 2/150
35/35 [==============================] - 0s 3ms/step - loss: 1.2094e-05 - accuracy: 1.0000
Epoch 3/150
35/35 [==============================] - 0s 3ms/step - loss: 3.9959e-06 - accuracy: 1.0000
Epoch 4/150
35/35 [==============================] - 0s 3ms/step - loss: 3.5400e-06 - accuracy: 1.0000
Epoch 5/150
35/35 [==============================] - 0s 2ms/step - loss: 2.9766e-06 - accuracy: 1.0000
Epoch 6/150
35/35 [==============================] - 0s 2ms/step - loss: 2.5723e-06 - accuracy: 1.0000
Epoch 7/150
35/35 [==============================] - 0s 3ms/step - loss: 2.2078e-06 - accuracy: 1.0000
Epoch 8/150
35/35 [==============================] - 0s 3ms/step - loss: 2.0061e-06 - accuracy: 1.0000
Epoch 9/150
35/35 [==============================] - 0s 3ms/step - loss: 1.6307e-06 - accuracy: 1.0000
Epoch 10/150
35/35 [==============================] - 0s 3ms/step - loss: 1.4

In [8]:
alfabeto = ['A','B', 'Nop']
import cv2
import numpy as np
from pickle import load
import HandTrackingModule as htm
import pandas as pd

normalizer = load(open('normalizerModel.pkl', 'rb'))
detector = htm.handDetector()
cap = cv2.VideoCapture(0)
formattedLm = []

rightMovesCounter = 0

while True:
    success, img = cap.read()
    img = detector.findHands(img, draw=True)

    sizes = detector.getNewSizes(img)
    cropped = img[sizes[0]:sizes[1], sizes[3]:sizes[2]]

    lmList = detector.findPositionWithoudId(img, draw=False)


    k = cv2.waitKey(1)

    
    if(cropped.shape[0] > 0 and cropped.shape[1] > 0):
        lmList = detector.findPosition(cropped, draw=False)
        imageHeight = cropped.shape[0]
        imageWidth = cropped.shape[1]
        comparisons = []

        for i, lm in enumerate(lmList):
            currentLmComparison = []
            for index in range(len(lmList)):
                if index == len(lmList):
                    break
                if lmList[i][0] == lmList[index][0]:
                    continue
                comparisonX = round(abs(((lmList[index][1] - lmList[i][1]) * 100) / imageWidth ), 2)
                comparisonY = round(abs(((lmList[i][2] - lmList[index][2]) * 100) / imageHeight ), 2)
                currentLmComparison.append([lmList[i][0], lmList[index][0], comparisonX, comparisonY])
            comparisons.append(currentLmComparison)
        formattedLm = []
        for comparison in comparisons:
            for lm in comparison:
                formattedLm.append(lm[2])
                formattedLm.append(lm[3])
        x_normalized = pd.DataFrame(formattedLm).transpose()
        x_normalized = normalizer.transform(x_normalized)
        predict = modelo.predict(x_normalized, verbose =0)
        if(int(np.argmax(predict))):
            rightMovesCounter = 0
        if(predict[0][np.argmax(predict)] > 0.98 and int(np.argmax(predict)) < 2):
            rightMovesCounter += 1
            if(rightMovesCounter > 10):
                # TODO - Verificar o movimento
                print("Opa mandou muito")
                break
        cv2.putText(img, alfabeto[int(np.argmax(predict))], (10, 70), cv2.FONT_HERSHEY_PLAIN, 3,(255, 0, 255), 3)
        cv2.putText(img, 'Acc: ' + str(predict[0][np.argmax(predict)]), (80, 70), cv2.FONT_HERSHEY_PLAIN, 3,(255, 0, 255), 3)
    else:
       rightMovesCounter=0
    if k%256 == 27:
        print("Escape hit, closing...")
        break
    cv2.imshow("Image", img)



cap.release()
cv2.destroyAllWindows()

Opa mandou muito
